In [6]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import Config.constants as cnst
import Config.experiment_config as cnfg

from DataSetLoaders.Lund2013DataSetLoader import Lund2013DataSetLoader as Lund2013
from GazeDetectors.EngbertDetector import EngbertDetector
from GazeDetectors.NHDetector import NHDetector

## Load Data Set

In [7]:
lund_dataset = Lund2013().download()

trial1 = lund_dataset[lund_dataset[cnst.TRIAL] == 2]
t, x, y = trial1[cnst.MILLISECONDS].to_numpy(), trial1[cnst.X].to_numpy(), trial1[cnst.Y].to_numpy()
e = trial1["MN"].to_numpy()

## Detect Events

In [8]:
engbert = EngbertDetector()
engbert_results = engbert.detect(t, x, y)

nh = NHDetector()
nh_results = nh.detect(t, x, y)

### Figure 1: Gaze Locations
Scatter plot of X and Y coordinates, colored by each sample's event (pre-annotated) 

In [9]:
events = trial1["MN"]
colors = np.array([np.array([cnfg.EVENT_MAPPING[val]["color"] for val in events])])

# TODO: add background image

fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', marker=dict(color=e, size=5)))
fig.show()

### Figure 2: Gaze over Time
Presenting the X- and Y-coordinates over time, with the velocity and/or acceleration (normalized to avoid scale issues).
Bottom of the figure will present the detected events.

In [20]:
# normalize velocity & acceleration
v, a = nh_results[cnst.GAZE][cnst.VELOCITY], nh_results[cnst.GAZE][cnst.ACCELERATION]
nv = v / np.nanmax(v)
na = a / np.nanmax(a)

# create a discrete color scale for the heatmap
# see https://chart-studio.plotly.com/%7Eempet/15229/heatmap-with-a-discrete-colorscale/#/
event_vals = [e.value for e in cnst.EVENTS]
event_vals.extend([len(cnst.EVENTS)])
normalized_event_vals = [(v - event_vals[0]) / (event_vals[-1] - event_vals[0]) for v in event_vals]
discrete_colors = []
for k in range(len(cnst.EVENTS)):
    discrete_colors.extend([(normalized_event_vals[k], cnfg.EVENT_MAPPING[cnst.EVENTS(k)]["color"]),
                            (normalized_event_vals[k + 1], cnfg.EVENT_MAPPING[cnst.EVENTS(k)]["color"])])

# create the figure with two y-axes
fig = make_subplots(specs=[[{"secondary_y": True}]])

# add lines for gaze location & velocity
fig.add_trace(go.Scatter(x=t, y=x, mode="lines", line=dict(color='#ff0000', width=4), name="X"), secondary_y=False)
fig.add_trace(go.Scatter(x=t, y=y, mode="lines", line=dict(color='#0000ff', width=4), name="Y"), secondary_y=False)
fig.add_trace(go.Scatter(x=t, y=nv, mode="lines", line=dict(color='#888888', width=2, dash='dash'), name="v"), secondary_y=True)
# fig.add_trace(go.Scatter(x=t, y=na, mode="lines", line=dict(color='#888888', width=2, dash='dot'), name="a"), secondary_y=True)

# add events as heatmap
line_size = 0.5 * np.nanmin([x, y])
fig.add_trace(go.Heatmap(z=[events], zmin=np.nanmin(event_vals), zmax=np.nanmax(event_vals),
                         x=t, y=[0, line_size],
                         colorscale=discrete_colors,
                         colorbar=dict(
                             len=0.5,
                             thickness=25,
                             tickvals=[np.mean(event_vals[k:k+2]) for k in range(len(event_vals)-1)],
                             ticktext=[e.name for e in cnst.EVENTS]
                         )),
              secondary_y=False)

# move legend to top left
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01
))

fig.show()

### Figure 3: Gaze over Time with Multiple Detectors
Adding the detection results for NHDetector and EngbertDetector to the previous figure.

In [21]:
nh_events = nh_results[cnst.GAZE][cnst.EVENT_TYPE]
eng_events = engbert_results[cnst.GAZE][cnst.EVENT_TYPE]

new_fig = go.Figure(fig)

fig.add_trace(go.Heatmap(z=[nh_events], zmin=np.nanmin(event_vals), zmax=np.nanmax(event_vals),
                         x=t, y=[-2 * line_size, -1 * line_size],
                         colorscale=discrete_colors, showscale=False,), 
              secondary_y=False)

fig.add_trace(go.Heatmap(z=[eng_events], zmin=np.nanmin(event_vals), zmax=np.nanmax(event_vals),
                         x=t, y=[-4 * line_size, -3 * line_size],
                         colorscale=discrete_colors, showscale=False,), 
              secondary_y=False)

fig.show()